# El juego del tres en raya

## Instalando dependencias

In [ ]:
!pip install stable-baselines3[extra]

## El gimnasio

In [ ]:
import numpy as np
import gym
from gym import spaces
import random


class tresEnRaya(gym.Env):

  def __init__(self, columns=3, rows=3):
    self.columns = columns
    self.rows = rows
    self.tablero = np.zeros(9, np.uint8)
    self.done = False
    self.turno = 0

    # Definimos el número de acciones que queremos realizar
    self.action_space = spaces.Discrete(9)

    # Vamos a definir el entorno
    self.observation_space = spaces.Box(
        low=0, 
        high=2,
        shape=(9,), 
        dtype=np.uint8)
    

  def __turno_aleatorio(self):
    colocada = 0
    while colocada == 0:
      alea = random.randint(0, 8)
      if self.tablero[alea] == 0:
        self.tablero[alea] = 2
        colocada = 1
    return self.tablero

  def __comprobar_ganador(self):
    # Comprobamos si gana la IA ha ganado
    # Horizontales
    if self.tablero[0] == 1 and self.tablero[1] == 1 and self.tablero[2] == 1:
      self.done = True
      return 1
    if self.tablero[3] == 1 and self.tablero[4] == 1 and self.tablero[5] == 1:
      self.done = True
      return 1
    if self.tablero[6] == 1 and self.tablero[7] == 1 and self.tablero[8] == 1:
      self.done = True
      return 1
    # Verticales
    if self.tablero[0] == 1 and self.tablero[3] == 1 and self.tablero[6] == 1:
      self.done = True
      return 1
    if self.tablero[1] == 1 and self.tablero[4] == 1 and self.tablero[7] == 1:
      self.done = True
      return 1
    if self.tablero[2] == 1 and self.tablero[5] == 1 and self.tablero[8] == 1:
      self.done = True
      return 1
    # Diagonales
    if self.tablero[0] == 1 and self.tablero[4] == 1 and self.tablero[8] == 1:
      self.done = True
      return 1
    if self.tablero[2] == 1 and self.tablero[4] == 1 and self.tablero[6] == 1:
      self.done = True
      return 1

    # Comprobamos si ha ganado el aleatorio
    # Horizontales
    if self.tablero[0] == 2 and self.tablero[1] == 2 and self.tablero[2] == 2:
      self.done = True
      return -1
    if self.tablero[3] == 2 and self.tablero[4] == 2 and self.tablero[5] == 2:
      self.done = True
      return -1
    if self.tablero[6] == 2 and self.tablero[7] == 2 and self.tablero[8] == 2:
      self.done = True
      return -1
    # Verticales
    if self.tablero[0] == 2 and self.tablero[3] == 2 and self.tablero[6] == 2:
      self.done = True
      return -1
    if self.tablero[1] == 2 and self.tablero[4] == 2 and self.tablero[7] == 2:
      self.done = True
      return -1
    if self.tablero[2] == 2 and self.tablero[5] == 2 and self.tablero[8] == 2:
      self.done = True
      return -1
    # Diagonales
    if self.tablero[0] == 2 and self.tablero[4] == 2 and self.tablero[8] == 2:
      self.done = True
      return -1
    if self.tablero[2] == 2 and self.tablero[4] == 2 and self.tablero[6] == 2:
      self.done = True
      return -1
    return 0.5
    
  def reset(self):
    """
    Importante: the observation must be a numpy array
    :return: (np.array) 
    """
    self.tablero = np.zeros(9, np.uint8)
    observation = np.zeros(9, np.uint8)
    self.turno = 0
    self.done = False
    return observation 
    
  def step(self, action):
    if self.tablero[action] != 0:
      observation = self.tablero
      self.done = True
      reward = -1
      info = {"Error": "Intento hacer trampa"}
      return observation, reward, self.done, info
    self.tablero[action] = 1
    self.turno += 1
    observation = self.tablero
    reward = self.__comprobar_ganador()
    if self.turno == 5:
      self.done = True
    info = {}
    if self.done == False:
      self.tablero = self.__turno_aleatorio()
      observation = self.tablero
    return observation, reward, self.done, info
 
  def render(self, mode='human'):
    print("")
    print("turno: " + str(self.turno))
    for i in range(9):
      if self.tablero[i]==0:
        print("   |", end="")
      elif self.tablero[i]==1:
        print(" O |",end="")
      elif self.tablero[i]==2:
        print(" X |",end="")
      if (i+1) % 3 == 0:
        print("")
    print("")

  


# Validamos el entorno

In [ ]:
from stable_baselines3.common.env_checker import check_env
env = tresEnRaya()
check_env(env)

## Probar el entorno

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.ppo.policies import MlpPolicy

env = tresEnRaya()
model = PPO(MlpPolicy, env, verbose=0).learn(int(50000))

In [ ]:
model.save("PPO5millonesFinal")

In [ ]:
env = tresEnRaya()
recompensas_finales = []
num_episodes = 5
for i in range(num_episodes):
  recompensas_episodio = []
  done = False
  obs = env.reset()
  while not done:
    action, _ = model.predict(obs)
    obs, reward, done, info = env.step(action)
    # env.render()
    recompensas_episodio.append(reward)
  env.render()
  print(info)
  recompensas_finales.append(sum(recompensas_episodio))
  # print("Fin")